In [ ]:
import os
os.nice(20)

# Add local paths
import sys

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec
from uncertainties import ufloat
from scipy.optimize import curve_fit 
import scipy.stats as stats 

import warnings
warnings.filterwarnings("ignore")

<h1>Define Binning</h1>

In [ ]:
bins = np.arange(0, 20, 1)
bins_mid = np.convolve(bins, [0.5, 0.5], "valid")

print(bins)
print(bins_mid)

<h1>Neutrino + Cosmics Bkg After Cosmics Rejection Scaled To POT</h1>

In [ ]:
#Should be the same for everyone I hope

bkg_before = [  25091.3327539,    28673.76052305,   43387.66703783,   93700.98298599,
              210912.74584594,  449691.77069344,  825255.86844471, 1306693.00107439,
              1710863.75968238, 1815966.66125165, 1556455.13817569, 1092645.54026722,
              658378.43989324,  345023.14695755, 186059.02132054,   97581.53701632,
              57098.51017194,   37964.24568798,   25471.19132599]

bkg_before_sum = np.sum(bkg_before)

print("Total Events = {}".format(bkg_before_sum))

#plot MC
plt.step(bins, np.insert(bkg_before, 0,0), label = 'Neutrino After Cosmic Rejection')
plt.show()

<h1>Neutrino + Cosmics Bkg After Full Selection Scaled To POT</h1>

In [ ]:
#Put your own here

bkg_after = [ 0., 0., 8.25366934, 9.5478061, 80.99813227,
            287.56878654, 555.30868766, 956.14925874, 1360.77696981,
            1340.44373184, 1027.43492706, 590.55131538, 344.0990943,   
            103.23512962, 64.23861731, 12.1360796, 10.84194285, 12.38050402, 5.42097142]

bkg_after_sum = np.sum(bkg_after)

print("Total Events = {}".format(bkg_after_sum))

#plot MC
plt.step(bins, np.insert(bkg_after, 0,0), label = 'Neutrino After Cosmic Rejection')
plt.show()



<h1>Fit Background After Cosmic Rejection</h1>

<h3>Define Gaussians</h3>

In [ ]:
#Gaussian 
def gaus(X, A, mean, sigma, C):
    return A * np.exp( -( X - mean )**2 / ( 2 * sigma**2 ) ) + C

half_mean = 18.936 /2
print(half_mean)

#The peak itself doesn't actually center
#Gaussian at a fixed mean
def gaus_fixedmean(X, A, sigma, C):
    return A * np.exp( -( X - half_mean )**2 / ( 2 * sigma**2 ) ) + C



<h3>Fit</h3>

In [ ]:
#Use Normal Gaussian

prediction = [max(bkg_before), 10, 1.308,  0]

lower_bound = [-np.inf, -np.inf, -np.inf, 0]

upper_bound = [np.inf, np.inf, np.inf, np.inf]

param_bounds=(lower_bound, upper_bound)

parameters_before, covariance_before = curve_fit(gaus, bins_mid, bkg_before, p0 = prediction, bounds=param_bounds) 

sigma_parameters_before = np.sqrt(np.diag(covariance_before))
#=====================================================================#
#error
fitA_before = ufloat(parameters_before[0], sigma_parameters_before[0])
figmean_before = ufloat(parameters_before[1], sigma_parameters_before[1])
fitSigma_before = ufloat(parameters_before[2], sigma_parameters_before[2])
fitC_before = ufloat(parameters_before[3], sigma_parameters_before[3])

text_res = "Best fit parameters:\nA = {}\nmean = {}\nsigma = {}\nC = {}".format(fitA_before, figmean_before, fitSigma_before, fitC_before)

#prediction
predict_before = gaus(bins_mid, *parameters_before)
bound_upper = gaus(bins_mid, *(parameters_before + sigma_parameters_before))
bound_lower = gaus(bins_mid, *(parameters_before - sigma_parameters_before))

# Chi-2
chi2_before = np.sum((bkg_before - predict_before)**2 / predict_before)
text_chi2 = "\nuChi2/Ndf = %.2f/(%.0f bins + $.0f fit parameters)".format(chi2_before, len(bins_mid), len(parameters_before))

print(text_res)
print(text_chi2) 
#=====================================================================#
fig, ax = plt.subplots(1,1, figsize = (8,6))

#plot MC
plt.step(bins, np.insert(bkg_before, 0,0), label = text_res)

#plot Gaussian Prediction
plt.plot(bins_mid, predict_before, 'red')

# plotting the confidence intervals
plt.fill_between(bins_mid, bound_lower, bound_upper,color = 'black', alpha = 0.2)

plt.legend(loc='best')
plt.show()

In [ ]:
#see some weired skewing effects here
#should I care

diff= bkg_before - predict_before
plt.scatter(bins_mid, diff)

plt.ylabel("MC - Fit")

<h1>Fit Background After Full Rejection</h1>

<h3>Define Gaussians using parameter from previous fitting</h3>

In [ ]:
#Use sigma fit from preselection neutrino distribution
fixed_sigma = parameters_before[2]
print(fixed_sigma)

fixed_mean = parameters_before[1]
print(fixed_mean)

#fixed mean AND fixed sigma
#only float amplitude
def gaus_fixedmean_fixedsigma(X, A, C):
    return A * np.exp( -( X - fixed_mean )**2 / ( 2 * fixed_sigma**2 ) ) + C

#fixed mean AND fixed sigma
#only float amplitude
def gaus_fixedsigma(X, A, mean, C):
    return A * np.exp( -( X - mean )**2 / ( 2 * fixed_sigma**2 ) ) + C

<h3>Fit</h3>

In [ ]:
#Use Gaussian w/ Fixed Sigma --> KINDA BAD

prediction = [max(bkg_after), 10, 0]

lower_bound = [-np.inf, -np.inf, 0]

upper_bound = [np.inf, np.inf, np.inf]

param_bounds=(lower_bound, upper_bound)

parameters_after, covariance_after = curve_fit(gaus_fixedsigma, bins_mid, bkg_after, p0 = prediction, bounds=param_bounds) 

sigma_parameters_after = np.sqrt(np.diag(covariance_after))
#=====================================================================#
#error
fitA_after = ufloat(parameters_after[0], sigma_parameters_after[0])
figmean_after = ufloat(parameters_after[1], sigma_parameters_after[1])
fitC_after = ufloat(parameters_after[2], sigma_parameters_after[2])

text_res = "Best fit parameters:\nA = {}\nmean = {}\nFixed sigma = {}\nC = {}".format(fitA_after, figmean_after, fitSigma_before, fitC_after)

#prediction
predict_after = gaus_fixedsigma(bins_mid, *parameters_after)
bound_upper = gaus_fixedsigma(bins_mid, *(parameters_after + sigma_parameters_after))
bound_lower = gaus_fixedsigma(bins_mid, *(parameters_after - sigma_parameters_after))

# Chi-2
chi2_after = np.sum((bkg_after - predict_after)**2 / predict_after)
text_chi2 = "\nuChi2/Ndf = {}/({} bins + {} fit parameters)".format(chi2_after, len(bins_mid), len(parameters_after))

print(text_res)
print(text_chi2) 
#=====================================================================#
fig, ax = plt.subplots(1,1, figsize = (8,6))

#plot MC
plt.step(bins, np.insert(bkg_after, 0,0), label = text_res)

#plot Gaussian Prediction
plt.plot(bins_mid, predict_after, 'red')

# plotting the confidence intervals
plt.fill_between(bins_mid, bound_lower, bound_upper,color = 'black', alpha = 0.2)

plt.legend(loc='best')
plt.show()

In [ ]:
#see some weired skewing effects here
#should I care

diff= bkg_after - predict_after
plt.scatter(bins_mid, diff)

plt.ylabel("MC - Fit")

<h1>Compare MC to Gaussian </h1>

In [ ]:
predict = gaus_fixedsigma(bins_mid, *parameters_after)
predict_ub = gaus_fixedsigma(bins_mid, *(parameters_after + sigma_parameters_after))
predict_lb = gaus_fixedsigma(bins_mid, *(parameters_after - sigma_parameters_after))

predict = np.array(predict)
print(predict)

In [ ]:
#plot just first 4 bins

fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 2]}, figsize = (6, 6), sharex = True)

#plot top plots
ax1.scatter(bins_mid, bkg_after, label = 'MC')
ax1.scatter(bins_mid, predict, label = 'Gaussian Fit')
ax1.fill_between(bins_mid, predict_lb, predict_ub, color = 'black', alpha = 0.15, label ='Fit Uncertainty')


ax1.legend(loc='upper left')

#plot bottom plots
diff= bkg_after - predict
ax2.scatter(bins_mid, diff, label = 'MC - Fit')
ax2.axhline(y = 0, color = 'r', ls = '--')

ax2.legend(loc='upper left')

ax1.set_ylim(0, 40)
#ax2.set_ylim(-10, 5)

ax1.set_xlim(0, 4)
ax2.set_xlim(0, 4)

In [ ]:
#plot just first 4 bins

fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 2]}, figsize = (6, 6), sharex = True)

#plot top plots
ax1.scatter(bins_mid, bkg_after, label = 'MC')
ax1.scatter(bins_mid, predict, label = 'Gaussian Fit')
ax1.fill_between(bins_mid, predict_lb, predict_ub, color = 'black', alpha = 0.15, label ='Fit Uncertainty')


ax1.legend(loc='upper right')

#plot bottom plots
diff= bkg_after - predict
ax2.scatter(bins_mid, diff, label = 'MC - Fit')
ax2.axhline(y = 0, color = 'r', ls = '--')

ax2.legend(loc='upper left')

ax1.set_ylim(0, 40)
ax2.set_ylim(-10, 10)

ax1.set_xlim(15, 19)
ax2.set_xlim(15, 19)

<h1>Plot Gaussian Fractional Error </h1>

In [ ]:
frac_err_lb =(predict - predict_lb) /predict

print(frac_err_lb)

frac_err_ub =(predict_ub - predict) /predict

print(frac_err_ub)

In [ ]:
frac_err_lb

In [ ]:
#plot just first 4 bins

fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 2]}, figsize = (6, 6), sharex = True)

#plot top plots
#ax1.scatter(bins_mid, cv_after, label = 'MC')
ax1.scatter(bins_mid, predict, label = 'Gaussian Fit')
ax1.fill_between(bins_mid, predict_lb, predict_ub, color = 'black', alpha = 0.15, label ='Fit Uncertainty')


ax1.legend(loc='upper left')

#plot bottom plots
ax2.scatter(bins_mid, frac_err_lb)
ax2.axhline(y = 0, color = 'r', ls = '--')
ax2.set_ylabel("fractional error")
#ax2.legend(loc='upper left')

#ax1.set_ylim(-5, 10)
#ax2.set_ylim(-10, 100)

ax1.set_xlim(0, 4)
ax2.set_xlim(0,4)

In [ ]:
#plot just first 4 bins

fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 2]}, figsize = (6, 6), sharex = True)

#plot top plots
#ax1.scatter(bins_mid, cv_after, label = 'MC')
ax1.scatter(bins_mid, predict, label = 'Gaussian Fit')
ax1.fill_between(bins_mid, predict_lb, predict_ub, color = 'black', alpha = 0.15, label ='Fit Uncertainty')

ax1.legend(loc='upper right')

#plot bottom plots
ax2.scatter(bins_mid, frac_err_lb)
ax2.axhline(y = 0, color = 'r', ls = '--')
ax2.set_ylabel("fractional error")
#ax2.legend(loc='upper left')

#ax1.set_ylim(-5, 10)
#ax2.set_ylim(-10, 100)

ax1.set_xlim(15, 19)
ax2.set_xlim(15, 19)

<h1>Save the Gaussian Fit to Be Used As Background</h1>